In [9]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib as mpl
import csv

In [10]:
def conv2d(filters, kernel_size, padding='v', strides=1, **kwargs):
    padding = 'valid' if padding == 'v' else 'same'
    x, y = kernel_size.split('x')
    kernel_size = [int(x), int(y)]
    return keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, **kwargs)

def max_pool2d(pool_size='2x2', padding='v', strides=1):
    x, y = pool_size.split('x')
    pool_size = [int(x), int(y)]
    padding = 'valid' if padding == 'v' else 'same'
    return keras.layers.MaxPool2D(pool_size=pool_size, strides=strides, padding=padding)


In [32]:
class Stem(keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        #conv 32 3x3 2 v
        #conv 32 3x3 v
        #conv 64 3x3 s
        #max pool 3x3 v + conv 96 3x3 2 v
        #conv 64 1x1 s   / 
        #conv 64 7x1 s / 
        #conv 64 1x7 s / conv 64 1x1 s
        #conv 96 3x3 s + conv 96 3x3 s
        #conv 192 3x3 v / max pool 2 v
        
                                               #299x299x3
        self.conv1 = conv2d(32, '3x3', 'v', 2) #149x149x3
        self.conv2 = conv2d(32, '3x3', 'v', 1) #147x147x3
        self.conv3 = conv2d(64, '3x3', 's', 1) #147x147x3

        self.max_pool4_1 = max_pool2d('3x3', 'v', 2) #1
        self.conv4_2 = conv2d(32, '3x3', 'v', 2)
        
        self.concat5 = keras.layers.Concatenate(axis=-1)
        
        self.conv6_1 = conv2d(64, '1x1', 's', 1)
        self.conv6_2 = conv2d(64, '7x1', 's', 1)
        self.conv6_3 = conv2d(96, '1x7', 's', 1)
        self.conv6_4 = conv2d(96, '3x3', 'v', 1)

        self.conv7_1 = conv2d(64, '1x1', 's', 1)
        self.conv7_2 = conv2d(96, '3x3', 'v', 1)

        self.concat8 = keras.layers.Concatenate(axis=-1)

        self.conv9_1 = conv2d(192, '3x3', 'v', 2)
        self.max_pool9_2 = max_pool2d('2x2', 'v', 2)

        self.concat10 = keras.layers.Concatenate(axis=-1)
    
    def call(self, inputs):
        Z = inputs
        Z = self.conv1(Z)
        Z = self.conv2(Z)
        Z = self.conv3(Z)
        Z_1 = self.max_pool4_1(Z)
        Z_2 = self.conv4_2(Z)
        Z = self.concat5([Z_1, Z_2])

        Z_1 = self.conv6_1(Z)
        Z_1 = self.conv6_2(Z_1)
        Z_1 = self.conv6_3(Z_1)
        Z_1 = self.conv6_4(Z_1)

        Z_2 = self.conv7_1(Z)
        Z_2 = self.conv7_2(Z_2)

        Z = self.concat8([Z_1, Z_2])
        
        Z_1 = self.conv9_1(Z)
        Z_2 = self.max_pool9_2(Z)

        return self.concat10([Z_1, Z_2])

        

In [35]:
inputs = keras.layers.Input(shape=[299, 299, 3])
stem_module = Stem()
stem_outputs = stem_module(inputs)

model = keras.models.Model(inputs=inputs, outputs=stem_outputs)
model.summary()


Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 299, 299, 3)]     0         
_________________________________________________________________
stem_8 (Stem)                (None, 35, 35, 384)       601760    
Total params: 601,760
Trainable params: 601,760
Non-trainable params: 0
_________________________________________________________________


In [25]:
#exercise
inputs = keras.layers.Input(shape=[299, 299, 3])
outputs =  keras.layers.Conv2D(32, kernel_size=3, strides=2, padding='valid')(inputs)
outputs = keras.layers.Conv2D(32, kernel_size=3, padding='valid')(outputs)
outputs = keras.layers.Conv2D(32, kernel_size=3, padding='same')(outputs)
outputs1 = keras.layers.Conv2D(96, kernel_size=3, padding='valid', strides=2)(outputs)
outputs1 = keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='valid')(outputs)
model = keras.models.Model(inputs=inputs, outputs=outputs)
model.summary()


Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 299, 299, 3)]     0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 149, 149, 32)      896       
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 147, 147, 32)      9248      
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 147, 147, 32)      9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 73, 73, 32)        0         
Total params: 19,392
Trainable params: 19,392
Non-trainable params: 0
_________________________________________________________________


In [ ]:
class InceptionModule(keras.layers.Layer):
    def __init__(self, filters, **kwargs):
        super().__init__(**kwargs)
        filters_1x1 = filters[0:2] + filters[3:4] + filters[5:]
        filters_3x3 = filters[2]
        filters_5x5 = filters[4]
        self.conv1x1_1 = keras.layers.Conv2D(filters_1x1[0], kernel_size=1, strides=1, padding="same")
        self.conv1x1_2 = keras.layers.Conv2D(filters_1x1[1], kernel_size=1, strides=1, padding="same")
        self.conv1x1_3 = keras.layers.Conv2D(filters_1x1[2], kernel_size=1, strides=1, padding="same")
        self.conv1x1_4 = keras.layers.Conv2D(filters_1x1[3], kernel_size=1, strides=1, padding="same")

        self.conv3 = keras.layers.Conv2D(filters_3x3, kernel_size=3, strides=1, padding="same")
        self.conv5 = keras.layers.Conv2D(filters_5x5, kernel_size=3, strides=1, padding="same")

        self.max_pool = keras.layers.MaxPooling2D(pool_size=3, strides=1, padding="same")

    def call(self, inputs):
        Z_1 = self.conv1x1_1(inputs)

        Z_2 = self.conv1x1_2(inputs)
        Z_2 = self.conv3(Z_2)

        Z_3 = self.conv1x1_3(inputs)
        Z_3 = self.conv5(Z_3)

        Z_4 = self.max_pool(inputs)
        Z_4 = self.conv1x1_4(Z_4)

        return keras.layers.Concatenate(axis=-1)([Z_1, Z_2, Z_3, Z_4])

In [3]:
class ResNetUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters, 3, strides=strides, padding="samen", use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters, 3, strides=1, padding="same", use_bias=False),
            keras.layers.BatchNormalization()
        ]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                keras.layers.Conv2D(filters, kernel_size=1, strides=strides, padding="same", use_bias=False),
                keras.layers.BatchNormalization()
            ]


    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        
        return self.activation(skip_Z + Z)

In [ ]:
class SE_Block(keras.layers.Layer):
    def __init__(self, filters, ratio=16):
        self.global_avg_pool = keras.layers.GlobalAveragePooling2D()
        self.squeeze = keras.layers.Dense(filters//ratio, activation='relu')
        self.excitation = keras.layers.Dense(filters, activation='sigmoid')
    
    def call(self, inputs):
        Z = self.global_avg_pool(inputs)
        Z = self.squeeze(Z)
        return self.excitation(Z)

In [ ]:
class XceptionModule(keras.layers.Layer):
    def __init__(self, filter):
        self.depthwise = keras.layers.Conv2D(filter, kernel_size=3, strides=1, padding='same')
        self.pointwise = keras.layers.Conv2D(filter, kernel_size=1, strides=1, padding="same")
    
    def call(self, inputs):
        Z = inputs
        Z = self.depthwise(Z)
        Z = keras.layers.BatchNormalization()(Z)
        Z = self.pointwise(Z)
        return keras.layers.BatchNormalization()(Z)